<a href="https://colab.research.google.com/github/mmcday/TestTest/blob/main/Badge_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Program

##Load Data

In [ ]:
import pandas as pd
from datetime import datetime

address_index = pd.read_csv("Address Index - NF Export.csv", encoding='ISO-8859-1')
trademark_sheet = pd.read_csv("Trademark License Agreement for the Member Badge and Taglines (Responses).csv", encoding='ISO-8859-1')

##Prep Data

In [ ]:
################### Address Index Clean #######################
#IF [M04_M03_KEY] = '45bffb2f-0052-4c0d-88b1-383ba81bb400' THEN 1 ELSE 0 END

#{FIXED[M04_CST_KEY]: SUM([BadgeRecord])}

#IF [BadgeSum]>0 THEN 1 ELSE 0 END

################### Address Index Clean #######################

#Add 'BadgeRecord'
address_index['BadgeRecord'] = address_index['M04_M03_KEY'].apply(
    lambda x: 1 if x == '45bffb2f-0052-4c0d-88b1-383ba81bb400' else 0
)

#Aggregate 'BadgeRecord' by 'M04_CST_KEY'
badge_sum_df = address_index.groupby('M04_CST_KEY')['BadgeRecord'].sum().reset_index()
badge_sum_df.rename(columns={'BadgeRecord': 'BadgeSum'}, inplace=True)

#Merge back to calculate 'HasBadge'
address_index = address_index.merge(badge_sum_df, on='M04_CST_KEY', how='left')
address_index['HasBadge'] = address_index['BadgeSum'].apply(lambda x: 1 if x > 0 else 0)

#filter out all cst_keys not 45bffb2f-0052-4c0d-88b1-383ba81bb400
#address_index = address_index[address_index['M04_M03_KEY'] == '45bffb2f-0052-4c0d-88b1-383ba81bb400']

In [ ]:
# Aggregate Address Index by Email Address
address_index_agg = address_index.groupby('EML_ADDRESS').agg({
    'M04_M03_KEY': 'last',
    'ORG_CST_KEY': 'last',
    'IND_CST_KEY': 'last',
    'HasBadge': 'max',
    'MBR_EXPIRE_DATE': 'last'
}).reset_index()

# Aggregate Trademark Sheet by Email Address
trademark_agg = trademark_sheet.groupby('Email Address').agg({
    'Date Signed': 'max'
}).reset_index()

In [ ]:
################### Join & Clean #######################
#Join Address Index and Trademark Sheet by Matching Email Addresses

#[HasBadge]=0 AND [HasBadge]=/=NULL

#DATE([MBR_EXPIRE_DATE])>DATE("2023-01-01 01:00")

################### Join & Clean #######################

#Join Address Index and Trademark by Email Addresses
joined_df = pd.merge(
    trademark_agg,
    address_index_agg,
    left_on='Email Address',
    right_on='EML_ADDRESS',
    how='left')

#Filter rows
joined_df = joined_df[
    (joined_df['HasBadge'] == 0) & (joined_df['HasBadge'].notnull()) &
    (pd.to_datetime(joined_df['MBR_EXPIRE_DATE']) > pd.to_datetime("2023-01-01"))
]

In [ ]:
joined_df

,Email Address,Date Signed,EML_ADDRESS,M04_M03_KEY,ORG_CST_KEY,IND_CST_KEY,HasBadge,MBR_EXPIRE_DATE
60,abuinicki@cnnssa.org,4/2/2025 18:32,abuinicki@cnnssa.org,3a182bb4-383d-42aa-ab7c-a60d4971ad14,f1b5708c-c778-4481-9c5e-1e00dd4c5104,b82b9543-e9c2-40f0-8fb3-42dfbf762636,0.0,12/31/2025 0:00
296,director.gcseniors@outlook.com,3/18/2025 10:56,director.gcseniors@outlook.com,None,82eb842a-d587-4157-ab42-f4ad47bb6f66,0735d5ae-6ce7-4ca1-909b-e98d821a9ebe,0.0,12/31/2025 0:00
640,lrodriguez@mowo.org,4/4/2025 11:29,lrodriguez@mowo.org,None,20db2134-87fa-4f48-8a64-b038a8ace13e,d6b1ff76-1cc0-45e1-ac95-404b2aa143d8,0.0,12/31/2025 0:00


## Create Output

In [ ]:
################### Create Outputs #######################
##############Badges Update
#Create New df
#M04_03_Key = M04_M03_Key

#M03_ADD_User = *User who runs program* -MMcDay_MOWAA

#M04_ADD_Date = Today

#M04_Change_User = Null

#M04_Change_Date = Null

#M04_Delete_Flag = 0

#M04_Entity_Key = Null

#M04_Key = Null

#M04_CST_Key
#Rename from Org_Cst_Key

#M04_Join_Date
#Rename from Date Signed

################### Create Outputs #######################

#Create Badge Updates
badges_update = pd.DataFrame()
badges_update['M04_Key'] = None
badges_update['M04_03_Key'] = joined_df['M04_M03_KEY']
badges_update['M04_CST_Key'] = joined_df['ORG_CST_KEY']
badges_update['M03_ADD_User'] = "MMcDay_MOWAA"  # Whoever Updates
badges_update['M04_ADD_Date'] = datetime.now().strftime('%Y-%m-%d')
badges_update['M04_Change_User'] = None
badges_update['M04_Change_Date'] = None
badges_update['M04_Delete_Flag'] = 0
badges_update['M04_Entity_Key'] = None
badges_update['M04_Join_Date'] = joined_df['Date Signed']

In [ ]:
import uuid

#Create random GUID(uuid4) for all Badges
for index, row in badges_update.iterrows():
    guid = uuid.uuid4()
    badges_update.at[index, 'M04_Key'] = guid
    print(f"Generated GUID (UUID4): {guid}")

Generated GUID (UUID4): d0bbeb6b-cd34-491b-bde8-ad3781e8303e
Generated GUID (UUID4): 5d86c457-d467-433b-b711-24e3c5667d0a
Generated GUID (UUID4): 8faf907f-5318-4d49-9cc6-00c0f2827eb4


In [ ]:
badges_update

,M04_Key,M04_03_Key,M04_CST_Key,M03_ADD_User,M04_ADD_Date,M04_Change_User,M04_Change_Date,M04_Delete_Flag,M04_Entity_Key,M04_Join_Date
60,d0bbeb6b-cd34-491b-bde8-ad3781e8303e,3a182bb4-383d-42aa-ab7c-a60d4971ad14,f1b5708c-c778-4481-9c5e-1e00dd4c5104,MMcDay_MOWAA,2025-04-09,None,None,0,None,4/2/2025 18:32
296,5d86c457-d467-433b-b711-24e3c5667d0a,None,82eb842a-d587-4157-ab42-f4ad47bb6f66,MMcDay_MOWAA,2025-04-09,None,None,0,None,3/18/2025 10:56
640,8faf907f-5318-4d49-9cc6-00c0f2827eb4,None,20db2134-87fa-4f48-8a64-b038a8ace13e,MMcDay_MOWAA,2025-04-09,None,None,0,None,4/4/2025 11:29


In [ ]:
badges_update.to_excel('Badge Update.xlsx', sheet_name='badges', index=False)

#Debug

## General Debug

In [ ]:
filtered_df = address_index[(address_index['M04_M03_KEY'] == '9dbf76b9-3228-4faa-b897-2ed8f1cd95c3') & (address_index['M04_CST_KEY'] == '405911e3-82e4-477d-a9df-adfc16c15403')]
filtered_df

,IND_CST_KEY,ORG_CST_KEY,IND_LAST_NAME,EML_ADDRESS,ORG_NAME,IXO_RLT_CODE,IXO_TITLE,IXO_START_DATE,IXO_END_DATE,ADR_STATE,...,M04_ADD_USER,M04_ADD_DATE,M04_CHANGE_USER,M04_CHANGE_DATE,M04_DELETE_FLAG,M04_JOIN_DATE,MBR_EXPIRE_DATE,BadgeRecord,BadgeSum,HasBadge
6877,86289d20-2456-4c4d-8e41-2c96ddbb20b0,405911e3-82e4-477d-a9df-adfc16c15403,Pritchett,wendy.mow21@gmail.com,Montgomery County Free Clinic Inc.,Operations,Driver Coordinator,1/3/2023 0:00,NaN,IN,...,Ehood_MOWAA,3/2/2023 9:56,NaN,NaN,0.0,12/21/2022 0:00,12/31/2025 0:00,0,3.0,1
6878,86289d20-2456-4c4d-8e41-2c96ddbb20b0,405911e3-82e4-477d-a9df-adfc16c15403,Pritchett,wendy.mow21@gmail.com,Montgomery County Free Clinic Inc.,Operations,Driver Coordinator,1/3/2023 0:00,NaN,IN,...,DCook_MOWAA,1/14/2025 10:00,NaN,NaN,0.0,1/14/2025 0:00,12/31/2025 0:00,0,3.0,1
15675,bb85f300-9397-4c6d-8b22-69976fdf70fa,405911e3-82e4-477d-a9df-adfc16c15403,Menard,ashley.menard@mcfreeclinic.org,Montgomery County Free Clinic Inc.,Principal Member,Executive Director,10/28/2024 0:00,NaN,NaN,...,Ehood_MOWAA,3/2/2023 9:56,NaN,NaN,0.0,12/21/2022 0:00,12/31/2025 0:00,0,3.0,1
15676,bb85f300-9397-4c6d-8b22-69976fdf70fa,405911e3-82e4-477d-a9df-adfc16c15403,Menard,ashley.menard@mcfreeclinic.org,Montgomery County Free Clinic Inc.,Principal Member,Executive Director,10/28/2024 0:00,NaN,NaN,...,DCook_MOWAA,1/14/2025 10:00,NaN,NaN,0.0,1/14/2025 0:00,12/31/2025 0:00,0,3.0,1
25225,0add118e-e733-4782-8200-ad991996c0e6,405911e3-82e4-477d-a9df-adfc16c15403,Stull,eggman4549@gmail.com,Montgomery County Free Clinic Inc.,Program Leader,Program Director,12/15/2022 0:00,NaN,IN,...,Ehood_MOWAA,3/2/2023 9:56,NaN,NaN,0.0,12/21/2022 0:00,12/31/2025 0:00,0,3.0,1
25226,0add118e-e733-4782-8200-ad991996c0e6,405911e3-82e4-477d-a9df-adfc16c15403,Stull,eggman4549@gmail.com,Montgomery County Free Clinic Inc.,Program Leader,Program Director,12/15/2022 0:00,NaN,IN,...,DCook_MOWAA,1/14/2025 10:00,NaN,NaN,0.0,1/14/2025 0:00,12/31/2025 0:00,0,3.0,1


In [ ]:
filtered_df = address_index[(address_index['M04_CST_KEY'] == '45bffb2f-0052-4c0d-88b1-383ba81bb400') & (address_index['M04_M03_KEY'] == '405911e3-82e4-477d-a9df-adfc16c15403')]
filtered_df

,IND_CST_KEY,ORG_CST_KEY,IND_LAST_NAME,EML_ADDRESS,ORG_NAME,IXO_RLT_CODE,IXO_TITLE,IXO_START_DATE,IXO_END_DATE,ADR_STATE,...,M04_ADD_USER,M04_ADD_DATE,M04_CHANGE_USER,M04_CHANGE_DATE,M04_DELETE_FLAG,M04_JOIN_DATE,MBR_EXPIRE_DATE,BadgeRecord,BadgeSum,HasBadge


In [ ]:
filtered_df = address_index[(address_index['HasBadge'] == 1) & (address_index['M04_CST_KEY'] == '405911e3-82e4-477d-a9df-adfc16c15403')]
filtered_df

,IND_CST_KEY,ORG_CST_KEY,IND_LAST_NAME,EML_ADDRESS,ORG_NAME,IXO_RLT_CODE,IXO_TITLE,IXO_START_DATE,IXO_END_DATE,ADR_STATE,...,M04_ADD_USER,M04_ADD_DATE,M04_CHANGE_USER,M04_CHANGE_DATE,M04_DELETE_FLAG,M04_JOIN_DATE,MBR_EXPIRE_DATE,BadgeRecord,BadgeSum,HasBadge
6876,86289d20-2456-4c4d-8e41-2c96ddbb20b0,405911e3-82e4-477d-a9df-adfc16c15403,Pritchett,wendy.mow21@gmail.com,Montgomery County Free Clinic Inc.,Operations,Driver Coordinator,1/3/2023 0:00,NaN,IN,...,MMcDay_MOWAA,1/21/2025 0:00,NaN,NaN,0.0,1/14/2025 9:59,12/31/2025 0:00,1,3.0,1
6877,86289d20-2456-4c4d-8e41-2c96ddbb20b0,405911e3-82e4-477d-a9df-adfc16c15403,Pritchett,wendy.mow21@gmail.com,Montgomery County Free Clinic Inc.,Operations,Driver Coordinator,1/3/2023 0:00,NaN,IN,...,Ehood_MOWAA,3/2/2023 9:56,NaN,NaN,0.0,12/21/2022 0:00,12/31/2025 0:00,0,3.0,1
6878,86289d20-2456-4c4d-8e41-2c96ddbb20b0,405911e3-82e4-477d-a9df-adfc16c15403,Pritchett,wendy.mow21@gmail.com,Montgomery County Free Clinic Inc.,Operations,Driver Coordinator,1/3/2023 0:00,NaN,IN,...,DCook_MOWAA,1/14/2025 10:00,NaN,NaN,0.0,1/14/2025 0:00,12/31/2025 0:00,0,3.0,1
15674,bb85f300-9397-4c6d-8b22-69976fdf70fa,405911e3-82e4-477d-a9df-adfc16c15403,Menard,ashley.menard@mcfreeclinic.org,Montgomery County Free Clinic Inc.,Principal Member,Executive Director,10/28/2024 0:00,NaN,NaN,...,MMcDay_MOWAA,1/21/2025 0:00,NaN,NaN,0.0,1/14/2025 9:59,12/31/2025 0:00,1,3.0,1
15675,bb85f300-9397-4c6d-8b22-69976fdf70fa,405911e3-82e4-477d-a9df-adfc16c15403,Menard,ashley.menard@mcfreeclinic.org,Montgomery County Free Clinic Inc.,Principal Member,Executive Director,10/28/2024 0:00,NaN,NaN,...,Ehood_MOWAA,3/2/2023 9:56,NaN,NaN,0.0,12/21/2022 0:00,12/31/2025 0:00,0,3.0,1
15676,bb85f300-9397-4c6d-8b22-69976fdf70fa,405911e3-82e4-477d-a9df-adfc16c15403,Menard,ashley.menard@mcfreeclinic.org,Montgomery County Free Clinic Inc.,Principal Member,Executive Director,10/28/2024 0:00,NaN,NaN,...,DCook_MOWAA,1/14/2025 10:00,NaN,NaN,0.0,1/14/2025 0:00,12/31/2025 0:00,0,3.0,1
25224,0add118e-e733-4782-8200-ad991996c0e6,405911e3-82e4-477d-a9df-adfc16c15403,Stull,eggman4549@gmail.com,Montgomery County Free Clinic Inc.,Program Leader,Program Director,12/15/2022 0:00,NaN,IN,...,MMcDay_MOWAA,1/21/2025 0:00,NaN,NaN,0.0,1/14/2025 9:59,12/31/2025 0:00,1,3.0,1
25225,0add118e-e733-4782-8200-ad991996c0e6,405911e3-82e4-477d-a9df-adfc16c15403,Stull,eggman4549@gmail.com,Montgomery County Free Clinic Inc.,Program Leader,Program Director,12/15/2022 0:00,NaN,IN,...,Ehood_MOWAA,3/2/2023 9:56,NaN,NaN,0.0,12/21/2022 0:00,12/31/2025 0:00,0,3.0,1
25226,0add118e-e733-4782-8200-ad991996c0e6,405911e3-82e4-477d-a9df-adfc16c15403,Stull,eggman4549@gmail.com,Montgomery County Free Clinic Inc.,Program Leader,Program Director,12/15/2022 0:00,NaN,IN,...,DCook_MOWAA,1/14/2025 10:00,NaN,NaN,0.0,1/14/2025 0:00,12/31/2025 0:00,0,3.0,1


In [ ]:
#Check 0s and 1s for address_index
address_index['HasBadge'].unique()

array([0, 1])

In [ ]:
address_index2['HasBadge'].unique()

NameError: name 'address_index2' is not defined

In [ ]:
#Count how many rows in address index have value 1 in HasBadge
address_index2[address_index2['HasBadge'] == 1].shape[0]

In [ ]:
address_index[address_index['HasBadge'] == 1].shape[0]

In [ ]:
badge_sum_df

In [ ]:
address_index[(address_index['HasBadge'] == 1) & (address_index['M04_M03_KEY'] == '45bffb2f-0052-4c0d-88b1-383ba81bb400')]

In [ ]:
address_index.dtypes